# Generates a GraphViz diagram from DuckDB Explain Analyze output. 

# AST Drawing
1. Converts the query to a serialized format via "select json_serialize_sql($1::varchar)", [query_string]
2. The JSON is converted to [dot](https://graphviz.org/doc/info/lang.html)
3. The resulting dot is passed to the GraphViz dot executable
4. The resulting jpg is displayed in the notebook

# Explain Analyze Drawing
When %dql or %%dql is executed with the `explain analyze_draw` option:
1. `PRAGMA enable_profiling=json` is executed
2. A relation is created with: `r = connection.sql(query)`
3. r.explain(type="analyze") is called, which returns the JSON description of the Explain Analyze
4. The JSON tree is converted to [dot](https://graphviz.org/doc/info/lang.html)
5. The resulting dot is passed to the GraphViz dot executable
6. The resulting jpg is displayed in the notebook

# Installation / Prerequisites
- %pip install duckdb # 0.8.0 or higher
- %pip install magic_duckdb # this library
- %pip install graphviz # wrapper around the actual graphviz software
- manually install graphviz from https://graphviz.org/download/

# Note:
pip install graphviz does *not* install GraphViz, merely the wrapper library. 

In [ ]:
%load_ext magic_duckdb

In [ ]:
import graphviz
import pathlib

# This is optional, if it's in your PATH
graphviz.backend.dot_command.DOT_BINARY = pathlib.Path("c:\\Program files\\graphviz\\bin\\dot")  # type: ignore # noqa


In [ ]:
%%dql -e ast_draw

-- Draws the json_serialize_sql version of the SQL

select * from range(10)

In [ ]:
# Draw the Explain Analyze

query = "select * from (select * from range(105) t(x)) join (select * from range(10)) on true"

# magic_duckdb.extras.explain_analyze_graphviz.dot_path= "c:\\Program files\\graphviz\\bin\\dot.exe"
graphviz.set_jupyter_format('png')

r = %dql --jinja2 -e explain_analyze_draw {{query}}
display(r)

In [ ]:
%%dql -e explain_analyze_draw

-- As a cell magic

select * from range(10)

In [ ]:
# In Code, no magics

import duckdb
from magic_duckdb.duckdb_mode import DuckDbMode

ddm = DuckDbMode()

with duckdb.connect() as con:
    dot = ddm.execute(connection = con, query_string="select * from range(10) join (select * from range(5)) on true", explain_function="explain_analyze_draw")
    display(dot)